In [9]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from flowdec.nb import utils as nbutils 
from flowdec import data as fd_data
import pims
# import pims
from flowdec import restoration as fd_restoration
from flowdec import data as fd_data
from flowdec import psf as fd_psf
import dask
import dask.array as da
import tifffile as tf
from nd2reader import ND2Reader
# from pims import ND2_Reader as ND2Reader
# import aicsimageio.vendor.omexml as ome
import glob

In [10]:
# channels = ['miRFP670', 'mRuby3','FITC']
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1
    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

def Nd2meta2OMEXML(reader, project=False):
#     from apeer_ometiff_library import omexmlClass
    import aicsimageio.vendor.omexml as omexmlClass
    
    #Missing TODO:
    #<Image>,  Name = "ImageName"
    #Instrument ID and Detector ID and Objective 
#     frames.parser._raw_metadata.image_metadata[b'SLxExperiment'][b'wsCameraName']

    # Objective settings with Refractive Index
        #Pixels, 
            #Channel Color = RGB###, EmissionWavelength, Name of Channel.
            #Plane  ExposureTime, Position X, Y, Z (Z is easy as it's in nd2reader metadata)
    def writeplanes(pixel, SizeT=1, SizeZ=1, SizeC=1, order='TZCYX', verbose=False):

        if order == 'TZCYX':

            pixel.DimensionOrder = omexmlClass.DO_XYCZT
            counter = 0
            for t in range(SizeT):
                for z in range(SizeZ):
                    for c in range(SizeC):

                        if verbose:
                            print('Write PlaneTable: ', t, z, c),
                            sys.stdout.flush()

                        pixel.Plane(counter).TheT = t
                        pixel.Plane(counter).TheZ = z
                        pixel.Plane(counter).TheC = c
                        #check basically because of triggered acquisition the arrays shouldn't have the size of "channel"
                        pixel.Plane(counter).DeltaT = reader.get_timesteps()[counter//SizeC]
                        pixel.Plane(counter).PositionZ = nd2meta['z_coordinates'][counter//SizeC]
#                         pixel.Plane(counter).ExposureTime = 
#                         pixel.Plane(counter).PositionX =
#                         pixel.Plane(counter).PositionY = 
#                         pixel.Plane(counter).
                        counter = counter + 1
                        
    
        return pixel
    
    #make a metadata var
    nd2meta = reader.metadata

#     Series = nd2meta['fields_of_view'][-1]+1
    scalex = nd2meta['pixel_microns']
    scaley = scalex
    if not project:
#         scalez = round(nd2meta['z_coordinates'][1]-nd2meta['z_coordinates'][0],3)
#         scalez = frames.parser._raw_metadata.image_metadata[b'SLxExperiment'][b'ppNextLevelEx'][b''][b'ppNextLevelEx'][b''][b'uLoopPars'][b'dZStep']
        scalez = 0.5
    pixeltype = 'uint16'
    dimorder = 'TZCYX'
# print(a)
    omexml = omexmlClass.OMEXML()
#     omexml.image_count = 1
#     omexml.image_count = reader.sizes['v']
    #Try to find if PIMS outputs the filename somehow.
    omexml.image(0).Name = 'Mamede Lab Image'
#     for i in range(frames.sizes['t']):
    p = omexml.image(0).Pixels
    p.SizeX = frames.sizes['x']
#     p.SizeX = 2044
    p.SizeY = frames.sizes['y']
    p.SizeC = frames.sizes['c']
    p.SizeT = frames.sizes['t']
    if project:
        p.SizeZ = 1
    else:
        p.SizeZ = frames.sizes['z']
        
    p.PhysicalSizeX = np.float(scalex)
    p.PhysicalSizeY = np.float(scaley)
    if not project:
        p.PhysicalSizeZ = np.float(scalez)
    p.PixelType = pixeltype
    p.channel_count = frames.sizes['c']
    
    if project:
        p.plane_count = 1 * p.SizeT * p.SizeC #* SizeV
    else:
        p.plane_count = p.SizeZ * p.SizeT * p.SizeC #* SizeV


    #I am using separate files for each visit point
    #, if you want one tiff with all visit points (possibly good for panels) 
    #you will need to update this section
    
    if project:
        p = writeplanes(p, SizeT=p.SizeT, SizeZ=1, SizeC=p.SizeC, order=dimorder)
    else:
        p = writeplanes(p, SizeT=p.SizeT, SizeZ=p.SizeZ, SizeC=p.SizeC, order=dimorder)
    for c in range(p.SizeC):
        p.Channel(c).Name = nd2meta['channels'][c]
#         p.Channel(c).Color =
#         p.Channel(c).EmissionWavelength =
        if pixeltype == 'unit8':
            p.Channel(c).SamplesPerPixel = 1
        if pixeltype == 'unit16':
            p.Channel(c).SamplesPerPixel = 2
            
    p.populate_TiffData()
#     omexml.structured_annotations.add_original_metadata(omexmlClass.OM_SAMPLES_PER_PIXEL, str(p.SizeC))

    return omexml

def observer(img, i, *args):
    #mgs.append(img.max(axis=0))
    if i % 5 == 0:
        print('Observing iteration = {} (dtype = {}, max = {:.3f})'.format(i, img.dtype, img.max()))   
#config = tf.ConfigProto(device_count={'GPU': 1})
#algo = fd_restoration.RichardsonLucyDeconvolver(n_dims=acq.data.ndim, pad_min=[1, 1, 1], session_config=config).initialize()



def deconv(chunk):
    # note that algo and cropped_kernel are from global scope ... ugly
#     print("chunk shape", chunk.shape)
    tmp = algo.initialize().run(fd_data.Acquisition(data=chunk, kernel=psf)
#                                 , session_config=tflow.compat.v1.ConfigProto(
#                                 device_count={'GPU': 1}        
#                                 , GPUOptions={'allow_growth' : 4}
#                                         )
                                ,niter=100
                               )
    return tmp.data

In [11]:
# fname = 'igfp1_caruby5_continue001trigger003.nd2'
dirname = "/home/jmamede/Data/pqbp1/20200824/new/*.nd2"
filelist = glob.glob(dirname)
print(filelist)

frames = ND2Reader(filelist[0])
frames.sizes

metadata = frames.metadata
metadata['channels']

ydivide = 4
xdivide = 4
frames

['/home/jmamede/Data/pqbp1/20200824/new/C02.nd2', '/home/jmamede/Data/pqbp1/20200824/new/F02.nd2', '/home/jmamede/Data/pqbp1/20200824/new/C04_new.nd2', '/home/jmamede/Data/pqbp1/20200824/new/F05.nd2', '/home/jmamede/Data/pqbp1/20200824/new/C05_new.nd2']


<FramesSequenceND>
Axes: 6
Axis 'x' size: 2048
Axis 'y' size: 2044
Axis 'c' size: 4
Axis 't' size: 1
Axis 'z' size: 35
Axis 'v' size: 10
Pixel Datatype: <class 'numpy.float64'>

In [13]:
import tifffile as tf
psf647_fiji = tf.imread("/home/jmamede/Data/PSF/AF647_0.3.tif")
psfruby_fiji =  tf.imread('/home/jmamede/Data/PSF/mRuby3_0.3.tif')

psfgfp_fiji =  tf.imread('/home/jmamede/Data/PSF/green_0.3.tif')
psfdapi_fiji =  tf.imread('/home/jmamede/Data/PSF/DAPI_0.3.tif')

In [15]:
frames.iter_axes = 't'  # 't' is the default already
frames.bundle_axes = 'zyx'  # when 'z' is available, this will be default
# frames.iter_axes
channels =  frames.metadata['channels']
metadata = frames.metadata
xml = Nd2meta2OMEXML(frames)
prjxml = Nd2meta2OMEXML(frames, project=True)

psf =[]
#psf = fd_psf.GibsonLanni.load('/tmp/psf.json')

# args = [na=1.4, wavelength= 0.594 , size_z=  11, size_x=  2048, size_y=  2044,
#         m :60,ns =  1.333, ni =  1.5156, tg =  170,         
#         res_lateral =  0.108333333333333, res_axial =  0.3, pZ = 1]

# args = [{"na": 1.4, "wavelength": '0.594' ,
#          "size_z": 11, "size_x": 2048, "size_y": 2044,
#          "m" :60,"ns" : 1.333, "ni" : 1.5156, "tg" : 170,
#          "res_lateral" : 0.108333333333333, "res_axial" : 0.3,
#          "pZ" :1}]

#na = meta['NA']
na = 1.4
#ra = meta['refractive_index1']
ra = 1.5156
z = frames.sizes['z']
#     x = frames.sizes['x'] // 2
#     y = frames.sizes['y'] // 2
#     z = 26
x = 258
y = 258
res_z = xml.image().Pixels.PhysicalSizeZ
res_xy =  xml.image().Pixels.PhysicalSizeX

##Make better with automatic wavelength from metadata and a loop and a dictionary with the PSFs
psf = fd_psf.GibsonLanni(
    na = na, wavelength= 0.670 , size_z=  z, size_x=  x , size_y=  y
    ,m =60, ns =  1.333
    , tg =  170, tg0 = 170
    , ni0 =  ra, ni =  ra
    ,res_lateral =  res_xy , res_axial = res_z, 
    #pz = 1
)
psf647 = psf.generate()    

psf = fd_psf.GibsonLanni(
    na = na, wavelength= 0.620 , size_z=  z, size_x=  x , size_y=  y
    ,m =60, ns =  1.333
    , tg =  170, tg0 = 170
    , ni0 =  ra, ni =  ra
    ,res_lateral =  res_xy , res_axial = res_z, 
    #pz = 1
)
psfruby = psf.generate()

psf = fd_psf.GibsonLanni(
    na = na, wavelength= 0.535 , size_z=  z, size_x=  x , size_y=  y
    ,m =60, ns =  1.333
    , tg =  170, tg0 = 170
    , ni0 =  ra, ni =  ra
    ,res_lateral =  res_xy , res_axial = res_z, 
    #pz = 1
)
psfgfp = psf.generate()

psf = fd_psf.GibsonLanni(
    na = na, wavelength= 0.420 , size_z=  z, size_x=  x , size_y=  y
    ,m =60, ns =  1.333
    , tg =  170, tg0 = 170
    , ni0 =  ra, ni =  ra
    ,res_lateral =  res_xy , res_axial = res_z, 
    #pz = 1
)
psfdapi = psf.generate()  

In [18]:
from skimage import data
import napari
#res8 = (res_stack/256).astype('uint8')
with napari.gui_qt():
    viewer = napari.Viewer()
    viewer.add_image(psfgfp
                     , colormap = 'red'
                    )
#     viewer.add_image(psfgfp_fiji
#                      ,colormap = 'green'
#                     )

KeyboardInterrupt: 

In [22]:
metadata


{'height': 2044,
 'width': 2048,
 'date': datetime.datetime(2020, 8, 31, 17, 4, 48),
 'fields_of_view': range(0, 10),
 'frames': [0],
 'z_levels': range(0, 35),
 'z_coordinates': [4237.4,
  4237.56,
  4238.18,
  4238.42,
  4238.78,
  4238.9,
  4239.18,
  4239.4400000000005,
  4239.86,
  4240.06,
  4240.28,
  4240.82,
  4240.86,
  4241.42,
  4241.7,
  4241.9400000000005,
  4242.2,
  4242.4,
  4242.82,
  4243.12,
  4243.36,
  4243.76,
  4243.9,
  4244.3,
  4244.66,
  4244.76,
  4245.22,
  4245.4800000000005,
  4245.8,
  4246.04,
  4246.32,
  4246.66,
  4247.08,
  4247.32,
  4247.56,
  4237.6,
  4237.8,
  4238.24,
  4238.58,
  4238.86,
  4239.2,
  4239.36,
  4239.7,
  4240.04,
  4240.28,
  4240.56,
  4241.02,
  4241.28,
  4241.42,
  4241.76,
  4242.08,
  4242.58,
  4242.58,
  4242.86,
  4243.3,
  4243.58,
  4243.82,
  4244.16,
  4244.34,
  4244.8,
  4245.16,
  4245.4,
  4245.72,
  4246.0,
  4246.5,
  4246.4,
  4247.02,
  4247.12,
  4247.62,
  4247.72,
  4236.72,
  4236.9800000000005,
  42